In [4]:
from models import MNISTConvNet, MNISTConvNetTrainer, get_mnist_dataloader


train, val, test = get_mnist_dataloader(128)

# print some sample images from the training set with matplotlib
# import matplotlib.pyplot as plt
# fig, axs = plt.subplots(2, 5, figsize=(12, 3))
# for i, ax in enumerate(axs.flat):
#     ax.imshow(train.dataset[i][0].squeeze(), cmap='gray')
#     ax.set_title(f'Label: {train.dataset[i][1]}')

model   = MNISTConvNet()
#trainer = MNISTConvNetTrainer(model, train, val, device='mps')

#trainer.train(num_epochs=1)

In [6]:
from models import MNISTFederatedServer

server = MNISTFederatedServer(100, 0.001, model, train, val, test)
server.init_clients(3, 50, lr=0.001)
server.start_train()


Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 0, Test Loss: 2.4075560569763184, Test Acc: 0.1135
Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 1, Test Loss: 2.7175638675689697, Test Acc: 0.1137
Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 2, Test Loss: 3.1014833450317383, Test Acc: 0.1217
Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 3, Test Loss: 3.4418160915374756, Test Acc: 0.1382
Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 4, Test Loss: 3.728562355041504, Test Acc: 0.1489
Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 5, Test Loss: 3.9715113639831543, Test Acc: 0.1562
Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 6, Test Loss: 4.18160343170166, Test Acc: 0.1622
Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 7, Test Loss: 4.3664655685424805, Test Acc: 0.1656
Local Accuracy: 1.0
Local Accuracy: 1.0
Local Accuracy: 1.0
Round 8

KeyboardInterrupt: 